In [31]:
from pickle import TRUE
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from datetime import timedelta
import email.utils as eutils
import time
import math
# import streamlit as st
import eppy as eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
from eppy.runner.run_functions import runIDFs
import PySimpleGUI as sg
# from PIL import Image, ImageTk
import os
from eppy.results import readhtml # the eppy module with functions to read the html
from eppy.results import fasthtml
import subprocess
import os
from os import system
import pylatex

from pylatex import Document, PageStyle, Head, MiniPage, Foot, LargeText, \
            MediumText, LineBreak, simple_page_number
from pylatex.utils import bold
from pylatex import Section, Subsection, Tabular, Math, TikZ, Axis, \
    Plot, Figure, Matrix, Alignat
from pylatex.utils import italic

In [47]:
carbonDatabase = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Carbon Correction Database.csv')

carbonDatabase.head()

carbonMeasures = ['HP Replace 2', 'HP Replace 3']

carbonMeasureCost = []
k_dirMR = 0.02


for measure in range(carbonDatabase.shape[0]):
    if carbonDatabase['Name'][measure] in carbonMeasures:
        carbonMeasureCost.append((carbonDatabase['Cost'][measure], carbonDatabase['Year'][measure]))

['HP Replace 2', 'HP Replace 3']


In [30]:
carbonDatabase = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Carbon Correction Database.csv')

carbonDatabase.head()

carbonMeasures = ['HP Replace 2', 'HP Replace 3']

carbonMeasureCost = []
k_dirMR = 0.02


for measure in range(carbonDatabase.shape[0]):
    if carbonDatabase['Name'][measure] in carbonMeasures:
        carbonMeasureCost.append((carbonDatabase['Cost'][measure], carbonDatabase['Year'][measure]))

firstCost = (135000,0)
carbonMeasureCost.append(firstCost)
dirMR = carbonMeasureCost

c_dirMR = []
years = range(70)
for i in years:
        # c_dirMR = []
        year = i+1
        for row in dirMR:
            
            if row[1] == i:
                # print(row[1])
                c_dirMR.append(row[0]/((1+k_dirMR)**year))
            else:
                c_dirMR.append(0)

        print(c_dirMR)

# emCO2 = [(emCO2_firstCost,1),((8500*laborFraction*0.3),20),((8500*laborFraction*0.3),40),((8500*laborFraction*0.3),60)] 

[0, 0, 132352.94117647057]
[0, 0, 132352.94117647057, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 132352.94117647057, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [166]:
fname = 'C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Testing Files/Try-Except test/Name your batch of files_Chicago-MDW_Phius Precriptive_BA.idf'
iddfile = "C:/EnergyPlusV9-5-0/Energy+.idd"

def costBuilder(idf, name, type, lineItemType, itemName, objEndUse, costEach, costArea):
    idf.newidfobject('ComponentCost:LineItem',
        Name = name,
        Type = type,
        Line_Item_Type = lineItemType,
        Item_Name = itemName,
        Object_EndUse_Key = objEndUse,
        Cost_per_Each = costEach,
        Cost_per_Area = costArea
        )
    
def glazingBuilder(idf, name, uFactor, shgc):
    idf.newidfobject('WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM',
        Name = name,
        UFactor = uFactor,
        Solar_Heat_Gain_Coefficient = shgc
        )



idf1 = IDF(fname)
windows = idf1.idfobjects['WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM']
# windowTable = pd.DataFrame(columns=['Name', 'U-Factor','SHGC'])
# for window in windows:
#     newResultRow = pd.DataFrame([{'Name':window.Name,
#                                   'U-Factor':window.UFactor,
#                                   'SHGC':window.Solar_Heat_Gain_Coefficient
#                                   }])
#     windowTable= pd.concat([windowTable, newResultRow], axis=0, ignore_index=True)#, ignore_index=True)

# windowTable.to_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Window Database.csv')

# print(windows)

glazingSpecs = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Window Database.csv')

glazings = glazingSpecs.shape[0]

for item in range(glazings):
    glazingBuilder(idf1, glazingSpecs['Name'][item], glazingSpecs['U-Factor'][item],glazingSpecs['SHGC'][item])


In [156]:
fname = 'C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Testing Files/Try-Except test/Name your batch of files_Chicago-MDW_Phius Precriptive_BA.idf'
iddfile = "C:/EnergyPlusV9-5-0/Energy+.idd"


def constructionBuilder(idf, constructionName, constructionLayers):
    params = [x for x in constructionLayers]
    layers = {}
    count = 0
    for i,param in enumerate(params):
        count = count + 1
        layers['Layer_' + str(count)] = param

    layers.pop('Layer_1')
    idf.newidfobject('Construction',
    Name = str(constructionName),
    Outside_Layer = str(constructionLayers[0]),
    **layers)

# IDF.setiddname(iddfile)
idf1 = IDF(fname)



constructions = idf1.idfobjects['CONSTRUCTION']

# for construct in constructions:
#     newResultRow = pd.DataFrame([{'Name':construct.Name,
#                                 'Outside_Layer':construct.Outside_Layer, 
#                                 'Layer_2':construct.Layer_2, 
#                                 'Layer_3':construct.Layer_3, 
#                                 'Layer_4':construct.Layer_4, 
#                                 'Layer_5':construct.Layer_5, 
#                                 'Layer_6':construct.Layer_6,
#                                 'Layer_7':construct.Layer_7,
#                                 'Layer_8':construct.Layer_8,
#                                 'Layer_9':construct.Layer_9,
#                                 'Layer_10':construct.Layer_10}])
#     ConstructionTable= pd.concat([ConstructionTable, newResultRow], axis=0, ignore_index=True)#, ignore_index=True)

# ConstructionTable.to_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Construction Database.csv')

# constructionList = pd.read_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Construction Database.csv')

# constructions = constructionList.shape[0]

# for item in range(constructions):
    
#     layers = [constructionList['Outside_Layer'][item],
#     constructionList['Layer_2'][item],
#     constructionList['Layer_3'][item],
#     constructionList['Layer_4'][item],
#     constructionList['Layer_5'][item],
#     constructionList['Layer_6'][item],
#     constructionList['Layer_7'][item],
#     constructionList['Layer_8'][item],
#     constructionList['Layer_9'][item],
#     constructionList['Layer_10'][item]]

#     layerList = [x for x in layers if str(x) != 'nan']

#     constructionBuilder(idf1, constructionList['Name'][item],layerList)

# IDFconstructions = idf1.idfobjects['CONSTRUCTION']

# print(IDFconstructions)


[
CONSTRUCTION,
    Brick Wall,               !- Name
    M01 100mm brick;          !- Outside Layer
, 
CONSTRUCTION,
    Ext_Door1,                !- Name
    G05 25mm wood;            !- Outside Layer
, 
CONSTRUCTION,
    Thermal Mass,             !- Name
    G05 25mm wood;            !- Outside Layer
, 
CONSTRUCTION,
    Interior Floor,           !- Name
    Plywood (Douglas Fir) - 12.7mm,    !- Outside Layer
    F05 Ceiling air space resistance,    !- Layer 2
    G01a 19mm gypsum board;    !- Layer 3
, 
CONSTRUCTION,
    Exterior Slab UnIns,      !- Name
    M15 200mm heavyweight concrete;    !- Outside Layer
, 
CONSTRUCTION,
    Exterior Slab + 2in EPS,    !- Name
    EPS 2in,                  !- Outside Layer
    M15 200mm heavyweight concrete;    !- Layer 2
, 
CONSTRUCTION,
    Exterior Wall,            !- Name
    F11 Wood siding,          !- Outside Layer
    R-11 3.5in Wood Stud,     !- Layer 2
    G01a 19mm gypsum board;    !- Layer 3
, 
CONSTRUCTION,
    Interior Wall,     

In [138]:
fname = 'C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Testing Files/Try-Except test/Name your batch of files_Chicago-MDW_Phius Precriptive_BA.idf'
iddfile = "C:/EnergyPlusV9-5-0/Energy+.idd"

# IDF.setiddname(iddfile)
idf1 = IDF(fname)

def materialBuilder(idf, name, rough, thick, conduct, dense, heatCap):
    idf.newidfobject('Material',
        Name = str(name),
        Roughness = str(rough),
        Thickness = thick,
        Conductivity = conduct,
        Density = dense,
        Specific_Heat = heatCap
        )

# MaterialTable = pd.DataFrame(columns=['Name', 'Roughness', 'Thickness', 'Conductivity', 'Density', 'Specific_Heat'])
# materials = idf1.idfobjects["MATERIAL"]
# for material in materials:
#     newResultRow = pd.DataFrame([{'Name':material.Name,
#                                   'Roughness':material.Roughness,
#                                   'Thickness': material.Thickness,
#                                   'Conductivity':material.Conductivity,
#                                   'Density':material.Density,
#                                   'Specific_Heat': material.Specific_Heat}])
#     MaterialTable = pd.concat([MaterialTable, newResultRow], axis=0, ignore_index=True)#, ignore_index=True)

# MaterialTable.to_csv('C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Material_Database.csv')

# materials = pd.read_csv("C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/REVIVE2024/Databases/Material_Database.csv")
# materialList = materials.shape[0]

# for item in range(materialList):
#     materialBuilder(idf1, materials['Name'][item], materials['Roughness'][item], materials['Thickness'][item], materials['Conductivity'][item],
#                     materials['Density'][item], materials['Specific_Heat'][item])